In [ ]:
!pip install langchainhub
!pip install langchain-qdrant
!pip install langchain transformers accelerate sentence-transformers

In [14]:
!git clone https://github.com/aalto-ui/chi21adaptive.git

Cloning into 'chi21adaptive'...
remote: Enumerating objects: 70, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 70 (delta 17), reused 53 (delta 10), pack-reused 0
Receiving objects: 100% (70/70), 370.94 KiB | 563.00 KiB/s, done.
Resolving deltas: 100% (17/17), done.


In [13]:
import os 

def convert_files_to_txt(src_dir, dst_dir):
    # If the destination directory does not exist, create it.
    if not os.path.exists(dst_dir):
        os.makedirs(dst_dir)
    
    for root, dirs, files in os.walk(src_dir):
        for file in files:
            if not file.endswith('.jpg'):
                file_path = os.path.join(root, file)
                
                # Get the relative path to preserve directory structure
                rel_path = os.path.relpath(file_path, src_dir)
                
                # Create the same directory structure in the new directory
                new_root = os.path.join(dst_dir, os.path.dirname(rel_path))
                os.makedirs(new_root, exist_ok=True)
                
                try:
                    with open(file_path, 'r', encoding='utf-8') as f:
                        data = f.read()
                except UnicodeDecodeError:
                    try:
                        with open(file_path, 'r', encoding='latin-1') as f:
                            data = f.read()
                    except UnicodeDecodeError:
                        print(f"Failed to decode the file: {file_path}")
                        continue
                
                # Create a new file path with .txt extension
                new_file_path = os.path.join(new_root, file + '.txt')
                with open(new_file_path, 'w', encoding='utf-8') as f:
                    f.write(data)

# Call the function with the source and destination directory paths
convert_files_to_txt('chi21adaptive', 'converted_codebase')

In [15]:
from langchain_community.document_loaders import DirectoryLoader,TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

src_dir = "converted_codebase"
loader = DirectoryLoader(src_dir, show_progress=True, loader_cls=TextLoader)
repo_files = loader.load()
print(f"Number of files loaded: {len(repo_files)}")
#
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=150)
documents = text_splitter.split_documents(documents=repo_files)
print(f"Number of documents : {len(documents)}")

0it [00:00, ?it/s]

Number of files loaded: 0
Number of documents : 0


In [16]:
for doc in documents:
 old_path_with_txt_extension = doc.metadata["source"]
 new_path_without_txt_extension = old_path_with_txt_extension.replace(".txt", "")
 doc.metadata.update({"source": new_path_without_txt_extension})

In [17]:
from langchain_community.embeddings import OllamaEmbeddings
#from langchain_community.vectorstores import Qdrant
from langchain_qdrant import Qdrant

In [18]:
model_name = "mxbai-embed-large"
embeddings = OllamaEmbeddings(model=model_name,
 show_progress=True,
 )

In [19]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en-v1.5"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings":True}
embeddings = HuggingFaceBgeEmbeddings(model_name=model_name,
 model_kwargs=model_kwargs,
 encode_kwargs=encode_kwargs,
 )

/home/tom/WD/.venv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/home/tom/WD/.venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [9]:
#Index new docs to db

qdrant = Qdrant.from_documents(
 documents,
 embeddings,
 path="local_qdrant",
 collection_name="my_documents",
)

OllamaEmbeddings: 100%|███████████████████████████████████████████████████| 56/56 [00:35<00:00,  1.57it/s]


In [ ]:
#Restore db

qdrant = Qdrant.from_existing_collection(
 embeddings,
 path="local_qdrant",
 collection_name="my_documents",
)

In [7]:
def pretty_print_docs(documents):
    for doc in documents:
        print(doc.metadata)
        print(" - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - ")
        print(doc.page_content)


In [8]:
query = "whats policy network here?"
found_docs = qdrant.similarity_search(query)
#pretty_print_docs(found_docs)

OllamaEmbeddings: 100%|█████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.10it/s]


In [10]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_core.prompts.chat import HumanMessagePromptTemplate

# LLM from Ollama
local_model = "codellama"
llm = ChatOllama(model=local_model)


### Implementation 1 (directly from serach docs and not from retreiver)

In [11]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")
Prompt: ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])
chain = prompt | llm | StrOutputParser()
response = chain.invoke({"question":query,"context":found_docs})
print(response)

The policy network is a neural network architecture that is trained to predict the correct response for a given prompt or input. In this case, the policy network is being used to generate responses for the menu items in the context of a chatbot or virtual assistant. The policy network takes in the input (e.g. "What's your favorite snack?") and outputs a response (e.g. "I don't have a personal preference, but some people like potato chips.").


### Implementation 2 (using retreiver)

In [28]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

In [29]:
from langchain import hub



retriever = MultiQueryRetriever.from_llm(
    qdrant.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = hub.pull("rlm/rag-prompt")
Prompt: ChatPromptTemplate(input_variables=['context', 'question'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])


In [30]:
# Define templates for prompts
from langchain_core.runnables import RunnableLambda
from operator import itemgetter
from langchain.memory import ConversationBufferMemory
from typing import List, Tuple
from langchain.schema import format_document

#Initialte chat_history

chat_history = []


# Create a memory instance
memory = ConversationBufferMemory(
    return_messages=True, output_key="answer", input_key="question", memory_key="chat_history"
)

# Define steps for the chain
loaded_memory = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("chat_history"),
)



# Define templates for prompts
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""


ANSWER_PROMPT = ChatPromptTemplate.from_template(template)

DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")


def _format_chat_history(chat_history: List[Tuple]) -> str:
    buffer = ""
    for dialogue_turn in chat_history:
        human = "HumanMessage: " + dialogue_turn[0]
        ai = "AIMessage: " + dialogue_turn[1]
        buffer += "\n" + "\n".join([human, ai])
    return buffer




def _combine_documents(
    docs, document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)




standalone_question = {
    "standalone_question": {
        "question": lambda x: x["question"],
        "chat_history": lambda x: x["chat_history"],        
#        "chat_history": lambda x: _format_chat_history(x["chat_history"]),
    }
    | CONDENSE_QUESTION_PROMPT
    | llm
    | StrOutputParser(),
}

retrieved_documents = {
    "docs": itemgetter("standalone_question") | retriever,
    "question": lambda x: x["standalone_question"],
}

final_inputs = {
    "context": lambda x: _combine_documents(x["docs"]),
    "question": itemgetter("question"),
}

answer = {
    "answer": final_inputs | ANSWER_PROMPT | llm,
    "docs": itemgetter("docs"),
}

# Create the final chain by combining the steps
final_chain = loaded_memory | standalone_question | retrieved_documents | answer

In [31]:
#stream chain 7

input = """
whats the policy network for?
"""
inputs = {"question": input, "chat_history": chat_history}



chunks = []
chunks_answer = []
for chunk in final_chain.stream(inputs):
    chunks.append(chunk)
    if 'answer' in chunk:
        print(chunk['answer'].content, end='')
        chunks_answer.append(chunk['answer'].content)
    else:
        pass


#Below code might not work


# Save the conversation in memory
#generated_answer = chunks['answer']

from langchain.schema.messages import HumanMessage, AIMessage

chat_history.extend([
    HumanMessage(content=input),
    AIMessage(content=chunks_answer),
    #AIMessage(content=result["answer"].content),
])


# Load memory to see the conversation history
memory.load_memory_variables({})

#memory.save_context(inputs, {"answer": generated_answer.content})
memory.save_context(inputs, {"answer": chunks_answer})

OllamaEmbeddings: 100%|█████████████████████████████████████████████████████| 1/1 [00:00<00:00,  7.02it/s]


Based on the provided code, it appears that the policy network is used to guide the user's search behavior in a menu-based system. The `UserOracle` class defines various methods related to user behavior, such as reading and recalling items from a menu.

The `recall` method seems to be a key component of the policy network, as it takes into account the user's experience (activation points) and previous interactions to determine the probability of recalling an item. This suggests that the policy network is used to model the user's decision-making process when searching for items in the menu.

In other words, the policy network aims to predict the likelihood of a user recalling an item from memory based on their past experiences and preferences. This can be seen as a form of cognitive modeling, where the network attempts to simulate how users make decisions and recall information.